In [ ]:
import re
import os
print(os.getcwd())

In [ ]:
path_prepend = 'tex_tables/'

In [ ]:
### TABLE 4: COMPILING ORDINAL RANKINGS TABLE || output: jel_ranking_table_together.tex
pubs_list = ['AER', 'ECA', 'JPE', 'QJE', 'RES']
pub_full_name_dict = {
    'AER' : 'American Economic Review',
    'ECA' : 'Econometrica',
    'JPE' : 'Journal of Political Economy',
    'QJE' : 'Quarterly Journal of Economics',
    'RES' : 'Review of Economic Studies'
}
print(pubs_list)
for pub in pubs_list:
    with open(path_prepend + 'jel_ranking_table_' + pub + '.tex', 'r') as pub_tab:
        old_tex = pub_tab.read()
        pub_full_name  = pub_full_name_dict.get(pub)

        # CHANGING ALIGNMENT OF COLUMNS
        new_tex = old_tex.replace('ccccccc', 'lllllll')
        
        # CHANING POSITIONING OF TABLE WITHIN PAGES
        new_tex = re.sub(r'!htbp', 'h', new_tex)

        # ADDING A PUBLICATION-LEVEL HEADER 
        new_tex = re.sub(r'\\hline \n\\hline \\\\\[-1.8ex\] ', 
            r'\\hline \n\\hline \\\\[-1.8ex] \\multicolumn{7}{c}{Publication : \\textit{' + pub_full_name + r'}} \\\\\n \\cline{1-7} \\\\',
            new_tex)

        # CLEANING UP THE COLUMN HEADERS TO BE ONLY YEARS (removing underscores and year so that 'AER\_1991' becomes '1991)
        new_tex = re.sub(r'[A-Z]{3}\\_', '', new_tex)

        # WRAPPING THE ENTIRE TABLE IN AN ENVIRONMENT THAT SCALES THE TABLE TO THE TEXT-WIDTH OF THE PAGE
        new_tex = re.sub(r'\\label{} \n', r'\\label{} \n\\begin{adjustbox}{max width = \\textwidth}\n', new_tex)
        new_tex = re.sub(r'\\end{tabular} \n', r'\\end{tabular} \n\\end{adjustbox} \n', new_tex)

        # CREATE A NICE, CLEANED UP PUBLICATION LEVEL TABLE (STORED AS TEMP, WILL BE DELETED LATER)
        with open(path_prepend + 'jel_ranking_table_' + pub +'_temp.tex', 'w') as temp_output:
            temp_output.write(new_tex)

first_tex_str = open(path_prepend + 'jel_ranking_table_' + pubs_list[0] + '_temp.tex', 'r').read()

for i in range(1, len(pubs_list)): 
    print('EXTRACING AND INJECTING TEX FOR PUBLICATION: {}'.format(pubs_list[i]))

    with open(path_prepend + 'jel_ranking_table_' + pubs_list[i] + '_temp.tex', 'r') as temp_table_file:
        temp_table_tex = temp_table_file.read()

        # WORKING
        table_body_of_interest = re.search(r'(?s)(?<=lllllll} \n\\\\\[-1.8ex\]\\hline \n\\hline \\\\\[-1.8ex\]).*(?=\\hline)', temp_table_tex).group()
        table_body_of_interest = re.sub(r'\\', r'\\\\', table_body_of_interest)

    ## WE INSERT THE TABLE BODIES INTO A SINGLE TABULAR ENVIRONMENT
    if i == 1:
        # This branch does the first table insertion, using a lot of the formatting and preamble work done by stargazer for us (which we basically treat as a wrapper and inject subsequent bodies into)
        complete_table_tex = re.sub(r'\\hline \\\\\[-1.8ex\] \n\\end{tabular} \n\\end{adjustbox} \n\\end{table}', table_body_of_interest + r'\n\\end{tabular} \n\\end{adjustbox} \n\\end{table}', first_tex_str)
    elif i == len(pubs_list) - 1:
        # This branch does the last table-body insertion, making sure to cap off with an \end{tabular environment}
        complete_table_tex = re.sub(r'\\end{tabular}', table_body_of_interest + r'\\hline \n\\end{tabular}', complete_table_tex)
    else:
        # This branch is the standard insertion branch
        complete_table_tex = re.sub(r'\\end{tabular}', table_body_of_interest + r'\n\\end{tabular}', complete_table_tex)


## ADDING A TABLE CAPTION/TITLE:
complete_table_tex = re.sub(r'\\caption{}', r'\\caption{Ordinal ranking of most frequently (weighted) appearing JEL codes in the Top 5 for selected years}', complete_table_tex)

with open(path_prepend + 'jel_ranking_table_together.tex', 'w') as final_output:
    final_output.write(complete_table_tex)


In [ ]:
### TABLE 4: CLEANUP ABOVE || output: jel_ranking_table_together.tex
for pub in pubs_list:
    file_name = path_prepend + 'jel_ranking_table_' + pub + '_temp.tex'
    if os.path.exists(file_name):
        print('removing: {}'.format(file_name))
        os.remove(file_name)

In [185]:
### TABLE 3: FIXING UP IO SHARE BY JOURNAL TABLE || output: io_share_journal_by_year_cleaned.tex
pubs_list = ['AER', 'ECA', 'JPE', 'QJE', 'RES', 'RJE']
with open(path_prepend + 'io_share_journal_by_year.tex', 'r') as original_table:
    original_tex = original_table.read()
    # print(original_tex)

    ### CREATING A TABLE TITLE/CAPTION
    new_tex = re.sub(r'\\caption{}', r'\\caption{Share of publications that are IO-related, by year}', original_tex)

    # CHANING COLUMN NAMES
    for pub in pubs_list:
        new_tex = re.sub(r'& '+ pub + r' &', r'& Count &', new_tex)
        new_tex = re.sub(r'& '+ pub, r'&', new_tex)
    
    # WRAPPING THE ENTIRE TABLE IN AN ENVIRONMENT THAT SCALES THE TABLE TO THE TEXT-WIDTH OF THE PAGE
    new_tex = re.sub(r'\\label{} \n', r'\\label{} \n\\begin{adjustbox}{max width = \\textwidth}\n', new_tex)
    new_tex = re.sub(r'\\end{tabular} \n', r'\\end{tabular} \n\\end{adjustbox} \n', new_tex)

    # ADDING COLUMN HEADERS (PUB NAME)
    for rev_pub in pubs_list:
        if rev_pub == pubs_list[0]:
            new_tex = re.sub(r'\n & Count',
            r'& \\multicolumn{2}{c}{\\textit{'+rev_pub+r'}}\n & Count',
            new_tex)
        elif rev_pub == pubs_list[len(pubs_list)-1]:
            new_tex = re.sub(r'\n & Count',
            r'& \\multicolumn{2}{c}{\\textit{'+rev_pub+r'}}\\\\ \n & Count',
            new_tex)
        else:
            new_tex = re.sub(r'\n & Count',
                r'& \\multicolumn{2}{c}{\\textit{'+rev_pub+r'}}\n & Count',
                new_tex)
    # ADDING LINES UNDER COLUMN HEADERS:
    cline_string = r'\n'
    for i in range(0,len(pubs_list)):
        cline_string  = cline_string + r'\\cline{' + str(i*2 +2) + r'-' + str(i*2 +3) + r'}'

    cline_string = cline_string + r'\\\\ '
    new_tex = re.sub(r'\n & Count',
        cline_string + r'\n & Count',
        new_tex)

    with open(path_prepend + 'io_share_journal_by_year_cleaned.tex', 'w') as output:
        output.write(new_tex)

In [ ]:
### CLEANUP ABOVE || output: io_share_journal_by_year_cleaned.tex
pass

In [186]:
### TABLE 1:  FIXING UP JOURNAL ARTICLE COUNT TABLE || output: top5_publications_by_year_cleaned.tex
# JUST REPLACING A TITLE FOR THE TIME BEING
with open(path_prepend + 'top5_publications_by_year.tex', 'r') as original_table:
    original_tex = original_table.read()
    
    ## CREATING A TABLE TITLE/CAPTION
    new_tex = re.sub(r'\\caption{}', r'\\caption{Number of articles published in each of the Top 5 (and RAND), by year}', original_tex)

    with open(path_prepend + 'top5_publications_by_year_cleaned.tex', 'w') as output:
        output.write(new_tex)
    

In [ ]:
### TABLE 1: CLEANUP ABOVE || output: top5_publications_by_year_cleaned.tex
pass

In [188]:
### TABLE 2: FIXING UP IO SHARE BY YEAR TABLE|| output: top5_publications_by_year_cleaned.tex
# JUST REPLACING A TITLE FOR THE TIME BEING
with open(path_prepend + 'io_share_by_year.tex', 'r') as original_table:
    original_tex = original_table.read()
    
    ## CREATING A TABLE TITLE/CAPTION
    new_tex = re.sub(r'\\caption{}', r'\\caption{The count and share of Industrial Organization articles in the Top 5 (and RAND), by year}', original_tex)

    with open(path_prepend + 'io_share_by_year_cleaned.tex', 'w') as output:
        output.write(new_tex)
    

In [187]:
### TABLE 2: CLEANUP ABOVE|| output: top5_publications_by_year_cleaned.tex
pass